In [1]:
import torch
from random import shuffle
import numpy as np
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import pandas as pd

random.seed(0)
np.random.seed(0)
torch.manual_seed(0)
torch.cuda.manual_seed(0)
torch.backends.cudnn.deterministic = True

NameError: name 'random' is not defined

In [ ]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# device = torch.device('cpu')
LABELS = [    
    "JUMPING",
    "JUMPING_JACKS",
    "BOXING",
    "WAVING_2HANDS",
    "WAVING_1HAND",
    "CLAPPING_HANDS"

] 
x_train_path = "shuffled_x_train_data.csv"
y_train_path = "shuffled_y_train_data.csv"
x_test_path = "shuffled_x_test_data.csv"
y_test_path = "shuffled_y_test_data.csv"


In [27]:
column_names = ["j0_x",  "j0_y", "j1_x", "j1_y", "j2_x", "j2_y", "j3_x", "j3_y", "j4_x", "j4_y", "j5_x", "j5_y", "j6_x", "j6_y", "j7_x", "j7_y", "j8_x", "j8_y",
                "j9_x", "j9_y", "j10_x", "j10_y", "j11_x", "j11_y", "j12_x", "j12_y", "j13_x", "j13_y", 'j14_x', "j14_y", "j15_x", "j15_y", "j16_x", "j16_y", "j17_x", "j17_y"]


DATASET_PATH = "poses/"

X_train_path = DATASET_PATH + "X_train.txt"
X_test_path = DATASET_PATH + "X_test.txt"

y_train_path = DATASET_PATH + "Y_train.txt"
y_test_path = DATASET_PATH + "Y_test.txt"

x_train = pd.read_csv(X_train_path, sep=",", names=column_names, header=None, dtype=np.float32)
x_test = pd.read_csv(X_test_path, sep=",", names=column_names, header=None, dtype=np.float32)
y_train = pd.read_csv(y_train_path, names=["labels"], dtype=np.int_)
y_test = pd.read_csv(y_test_path, names=["labels"], dtype=np.int_)

In [28]:
y_train = y_train - 1
y_test = y_test - 1
y_train.shape

(22625, 1)

In [15]:
x_train = pd.read_csv(x_train_path, sep="\t", dtype=np.float32)
x_test = pd.read_csv(x_test_path, sep="\t", dtype=np.float32)
y_train = pd.read_csv(y_train_path, sep="\t", dtype=np.int_)
y_test = pd.read_csv(y_test_path, sep="\t", dtype=np.int_)

In [10]:
def chunker(x_seq, y_seq, seq_size, batch_size):
    for batch_pos in range(0, len(y_seq), batch_size):
        x_batch = list()
        for pos in range(batch_size):
            # print(x_seq.iloc[pos*seq_size:pos*seq_size + seq_size])
            x_batch.append(x_seq.iloc[pos*seq_size:pos*seq_size + seq_size].values)
        
        yield torch.tensor(x_batch), torch.flatten(torch.tensor(y_seq.iloc[batch_pos:batch_pos + batch_size].values))
        

In [35]:
# Fully connected neural network with one hidden layer
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(LSTM, self).__init__()
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)
    def forward(self, x):
        out, _ = self.lstm(x)  
        out = self.fc(out)
        out = out[:, -1, :]
        return out

In [32]:
# Hyper-parameters 
num_classes = 6
num_epochs = 10
train_batch_size = 125
test_batch_size = 1
learning_rate = 0.00001

input_size = 36
sequence_length = 32
hidden_size = 36
num_layers = 1

In [36]:
model = LSTM(input_size, hidden_size, num_layers, num_classes).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [41]:
# Train the model
n_total_steps = int(y_train.shape[0] / train_batch_size)

for epoch in range(num_epochs):
    correct = 0
    for i, (x_batch, y_batch) in enumerate(chunker(x_train, y_train, sequence_length, train_batch_size)):  
        y_batch = y_batch.to(device)
        x_batch = x_batch.to(device)
        
        # Forward pass
        outputs = model(x_batch)
        
        loss = criterion(outputs, y_batch)
        # print(loss.item())
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # max returns (value ,index)
        _, predicted = torch.max(outputs.data, 1)
        # print(predicted.shape)
        correct += (predicted == y_batch).sum().item()
        print((predicted == y_batch).sum().item())
        if (i+1) % 10 == 0:
            
            print (f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}')
    acc = 100.0 * correct / y_train.shape[0]
    print (f'Accuracy: {acc} %')
    # Test the model
    # In test phase, we don't need to compute gradients (for memory efficiency)
    # correct = 0
    # with torch.no_grad():
    #     correct = 0
    #     for i, (x_batch, y_batch) in enumerate(chunker(x_test, y_test, sequence_length, 125)):  
    #         # origin shape: [N, 1, 28, 28]
    #         # resized: [N, 28, 28]
    #         y_batch = y_batch.to(device)
    #         x_batch = x_batch.to(device)
    #         outputs = model(x_batch)
            
    #         # max returns (value ,index)
    #         _, predicted = torch.max(outputs.data, 1)
    #         correct += (predicted == y_batch).sum().item()
    #     acc = 100.0 * correct / y_test.shape[0]
    #     print(f'Accuracy: {acc} %')

0
0
0
0
0
0
0
0
0
0
Epoch [1/10], Step [10/181], Loss: 1.8813
0
0
0
0
0
0
0
0
0
4
Epoch [1/10], Step [20/181], Loss: 1.7880
5
5
5
5
5
5
5
5
5
5
Epoch [1/10], Step [30/181], Loss: 1.7644
5
5
5
5
5
5
5
5
5
5
Epoch [1/10], Step [40/181], Loss: 1.7628
5
5
5
5
5
5
5
5
23
23
Epoch [1/10], Step [50/181], Loss: 1.7367
23
23
23
23
23
23
23
23
23
23
Epoch [1/10], Step [60/181], Loss: 1.7359
23
23
23
23
23
23
23
23
23
23
Epoch [1/10], Step [70/181], Loss: 1.7340
23
23
23
23
24
28
28
28
28
28
Epoch [1/10], Step [80/181], Loss: 1.7305
28
29
29
29
29
7
0
0
0
0
Epoch [1/10], Step [90/181], Loss: 1.7328
0
0
0
0
0
0
0
0
0
0
Epoch [1/10], Step [100/181], Loss: 1.7321
0
0
0
0
0
0
0
0
0
0
Epoch [1/10], Step [110/181], Loss: 1.7306
0
0
0
0
0
0
0
0
0
0
Epoch [1/10], Step [120/181], Loss: 1.7290
0
0
0
52
91
91
91
91
91
91
Epoch [1/10], Step [130/181], Loss: 1.7020
91
91
91
91
91
91
91
91
91
91
Epoch [1/10], Step [140/181], Loss: 1.7008
91
91
91
91
91
91
91
91
91
91
Epoch [1/10], Step [150/181], Loss: 1.6991


KeyboardInterrupt: 